In [1]:
import torch
import torch.nn as nn
import pandas as pd
import os
from torchvision import datasets, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm
import numpy as np

In [2]:
class AlbumentationsTransform:
    def __init__(self, is_train: bool = True):
        # 공통 변환 설정: 이미지 리사이즈, 정규화, 텐서 변환
        common_transforms = [
            A.Resize(224, 224),  # 이미지를 224x224 크기로 리사이즈
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
            ToTensorV2()  # albumentations에서 제공하는 PyTorch 텐서 변환
        ]
        
        if is_train:
            # 훈련용 변환: 랜덤 수평 뒤집기, 랜덤 회전, 랜덤 밝기 및 대비 조정 추가
            self.transform = A.Compose(
                [
                    A.Rotate(limit=10),  # 최대 10도 회전
                    A.ElasticTransform(alpha=1, sigma=10, alpha_affine=10, p=0.5),
                    #A.Erosion(kernel=(1, 2), p=0.5), 
                    #A.Dilation(kernel=(1, 2), p=0.5), # module 'albumentations' has no attribute 'Dilation'
                    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                    A.MotionBlur(blur_limit=(3, 7), p=0.5)
                ] + common_transforms
            )
        else:
            # 검증/테스트용 변환: 공통 변환만 적용
            self.transform = A.Compose(common_transforms)

    def __call__(self, image) -> torch.Tensor:
        # 이미지가 NumPy 배열인지 확인
        if not isinstance(image, np.ndarray):
            raise TypeError("Image should be a NumPy array (OpenCV format).")
        
        # 이미지에 변환 적용 및 결과 반환
        transformed = self.transform(image=image)  # 이미지에 설정된 변환을 적용
        
        return transformed['image']  # 변환된 이미지의 텐서를 반환

In [3]:
class TransformSelector:
    """
    이미지 변환 라이브러리를 선택하기 위한 클래스.
    """
    def __init__(self, transform_type: str):

        # 지원하는 변환 라이브러리인지 확인
        if transform_type in ["torchvision", "albumentations"]:
            self.transform_type = transform_type
        
        else:
            raise ValueError("Unknown transformation library specified.")

    def get_transform(self, is_train: bool):
        
        # 선택된 라이브러리에 따라 적절한 변환 객체를 생성
        if self.transform_type == 'torchvision':
            transform = TorchvisionTransform(is_train=is_train)
        
        elif self.transform_type == 'albumentations':
            transform = AlbumentationsTransform(is_train=is_train)
        
        return transform

In [4]:
latent_dim = 100

# 데이터 로드
traindata_dir = "./data/train"
traindata_info_file = "./data/train.csv"
train_df = pd.read_csv(traindata_info_file)

# 클래스 리스트 추출
classes = train_df['class_name'].unique()
n_classes = len(classes)  # 총 클래스 수 (500개)

# 클래스별로 몇 개의 이미지를 생성할지 지정
images_per_class = 30
total_images = n_classes * images_per_class


# ResNet 블록 정의
class ResNetBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual  # residual connection
        out = self.relu(out)
        return out


# DCGAN Generator 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = 7  # 224x224 이미지를 생성할 것이므로 초기 크기는 7x7로 설정
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 1024 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(1024),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 3, kernel_size=3, stride=1, padding=1), # 여기서 출력 채널 수를 1에서 3으로 변경
            nn.Tanh()  # output은 -1에서 1 사이 값
        )

        self.res_block = ResNetBlock(1024)  # ResNet 블록 추가

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 1024, self.init_size, self.init_size)
        out = self.res_block(out)  # ResNet 블록 통과
        img = self.conv_blocks(out)
        return img


# DCGAN Discriminator 정의
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, kernel_size=4, stride=2, padding=1)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters))
            block.append(nn.LeakyReLU(0.2, inplace=True))
            block.append(nn.Dropout(0.2))  # Dropout 추가 (20% 확률로 비활성화)
            return block

        self.model = nn.Sequential(
            *discriminator_block(3, 64, bn=False),  # Change from 1 to 3 channels here
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()  # output은 0에서 1 사이 확률 값
        )
        
         # AdaptiveAvgPool2d를 사용해 (1, 1)로 크기 고정
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, img):
        validity = self.model(img)
        validity = self.adaptive_pool(validity)  # (batch_size, 1, 1, 1) 형태로 변환
        return validity.view(-1, 1)  # (batch_size, 1) 형태로 변환
        # return self.model(img).view(-1, 1)


# 학습에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "albumentations"
)
transforms_train = transform_selector.get_transform(is_train=True)


# 학습 데이터셋에서 데이터 로드
class SketchDataset(torch.utils.data.Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.df.iloc[idx, 1])
        image = Image.open(img_path).convert('L')  # 이미지를 로드하고 그레이스케일로 변환
        image = np.array(image)  # Convert PIL image to NumPy array    
        
        # Expand grayscale image to 3 channels
        image = np.stack([image] * 3, axis=-1)  # Shape becomes (224, 224, 3)
        
        if self.transform:
            image = self.transform(image)
            
        target = self.df.iloc[idx, 2]
        return image, target

# 데이터셋과 DataLoader 설정
train_dataset = SketchDataset(df=train_df, root_dir=traindata_dir, transform=transforms_train)

# collate_fn을 설정하여 문제 해결
def custom_collate_fn(batch):
    images, targets = zip(*batch)
    images = torch.stack(images, 0)  # 이미지 리스트를 하나의 텐서로 묶음
    targets = torch.tensor(targets)  # 타겟 리스트를 텐서로 변환
    return images, targets

dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)


# 모델 초기화
generator = Generator().cuda()
discriminator = Discriminator().cuda()

# 손실 함수 및 학습률 설정
adversarial_loss = nn.BCELoss().cuda()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.001, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0005, betas=(0.5, 0.999))

# 학습 과정
n_epochs = 100
generated_data = []  # 새롭게 생성된 데이터를 저장하기 위한 리스트

# 생성된 이미지를 저장할 폴더 설정
augment_dir = "./data/augment"
os.makedirs(augment_dir, exist_ok=True)

for epoch in range(n_epochs):
    for i, (imgs, targets) in tqdm(enumerate(dataloader)):

        real = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(0.9) # 1.0 대신 0.9 사용
        fake = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(0.0)

        real_imgs = imgs.cuda()
        
        # 생성자 학습
        optimizer_G.zero_grad()
        z = torch.normal(0, 1, (imgs.shape[0], latent_dim)).cuda()
        generated_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(generated_imgs), real) # * 10  # 가중치를 10배로
        g_loss.backward()
        optimizer_G.step()   

        # 판별자 학습
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), real)
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        
        # Update D less frequently
        if i % 2 == 0:  # D를 2번마다 1번 업데이트
            optimizer_D.step()
            
        done = epoch * len(dataloader) + i

    print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}]")

    
# 모델을 평가 모드로 설정
generator.eval()
    
# 각 클래스마다 정확히 30개의 이미지 생성
new_data = []
for class_name in tqdm(classes):
    class_dir = os.path.join(augment_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    
    # 클래스별 30개의 이미지를 생성합니다.
    for i in range(images_per_class):
        # 노이즈 생성
        z = torch.randn(1, latent_dim).cuda()
        
        # 이미지 생성
        generated_img = generator(z).detach().cpu()
        
        # 생성된 이미지 저장
        img_path = os.path.join(class_dir, f"augmented_{i}.JPEG")
        save_image(generated_img, img_path, normalize=True)

        # CSV에 저장할 이미지 경로 및 타겟 클래스 정보 추가
        target = train_df[train_df['class_name'] == class_name]['target'].values[0]
        new_data.append({'class_name': class_name, 'image_path': os.path.relpath(img_path, augment_dir), 'target': target})


# 기존 데이터와 생성된 데이터 합치기
generated_df = pd.DataFrame(new_data)
augmented_train_df = pd.concat([train_df, generated_df], ignore_index=True)

# 업데이트된 train.csv 파일 저장
augmented_train_df.to_csv("./data/augmented_train.csv", index=False)

print(f"총 생성된 이미지 수: {len(new_data)}")

0it [00:00, ?it/s]/tmp/ipykernel_255595/1208927114.py:172: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  real = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(0.9) # 1.0 대신 0.9 사용
118it [04:46,  2.43s/it]

[Epoch 0/100] [D loss: 0.185891] [G loss: 4.992872]



25it [01:06,  2.67s/it]


KeyboardInterrupt: 

In [2]:
import os
augment_dir = './data/augment/'
class_folders = [folder for folder in os.listdir(augment_dir) if os.path.isdir(os.path.join(augment_dir, folder))]
print(f"클래스 폴더 개수: {len(class_folders)}")

FileNotFoundError: [Errno 2] No such file or directory: './data/augment/'

In [3]:
augment_dir = './data/train/'
class_folders = [folder for folder in os.listdir(augment_dir) if os.path.isdir(os.path.join(augment_dir, folder))]
print(f"클래스 폴더 개수: {len(class_folders)}")

클래스 폴더 개수: 500
